<a href="https://colab.research.google.com/github/emgrimm42/NLP_Modules/blob/main/Grimm_Download_%C3%BCber_API_der_DDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download über API der DDB

Dieses Google Colab Notebook basiert auf den Forschungsdaten-Notebooks der Deutschen Nationalbibliothek, die unter folgendem Link verfügbar sind: https://github.com/Deutsche-Digitale-Bibliothek/ddblabs-summer-school-2024


## Suchindizes der Deutsche Digitale Bibliothek
Die Deutsche Digitale Bibliothek betreibt [Solr](https://solr.apache.org/guide/8_8/searching.html)-Suchindizes, die für die verschiedenen Funktionen der (Sub-) Portale benötigt werden. Das Zeitungsportal benutzt zwei Suchindizes. Eine weiterführende Dokumentation befindet sich hier: https://api.deutsche-digitale-bibliothek.de/#/search/getSolrSearch

- `newspaper`: enthält Informationen über Zeitungstitel
  - Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/schema.xml
  - Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper/conf/solrconfig.xml
- `newspaper-issues`: enthält die zeitungsbezogenen Metadaten inkl. Volltexte
  - Schema: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/schema.xml
  - Konfiguration: https://dev.fiz-karlsruhe.de/stash/projects/DDB/repos/ddb-backend/browse/Cortex/conf/solr/newspaper-issues/conf/solrconfig.xml

## Suchindex `newspaper`
Der Suchindex `newspaper` ist ein Suchindex über alle Zeitungstitel der [Zeitschriftendatenbank (ZDB)](https://zdb-katalog.de/). Im Schema des Suchindex (s.o.) sind die Suchfelder (Facetten) dokumentiert. Wenn Zeitungstitel gefunden werden sollen, die im Zeitungsportal verfügbar sind, dann muss nach `hasLoadedIssues:true` (Feld:Wert) gesucht werden.

- https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true

Die Suchfelder können beliebig miteinander kombiniert werden. Das geht mit den Operatoren `AND` und `OR`. Möchte man beispielsweise Zeitungen mit „La otra Alemania“ im Titel suchen, die auch im Zeitungsportal verfügbar sind, dann kombiniert man: `hasLoadedIssues:true AND title:"La otra Alemania"`

- [https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true AND title:"La otra Alemania"](https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper/select?q=hasLoadedIssues:true%20AND%20title:"La+otra+Alemania")

### Python-Programmcode

Python-Programmcode in Jupyter Notebooks ermöglicht interaktives Programmieren und sofortige Anzeige von Ergebnissen. Es ist ideal für die Datenanalyse und Visualisierungen. Python-Bibliotheken können in Notebooks nachgenutzt werden und erhöhen so den Funktionsumfang.

Die o. g. Solr-Abfrage kann mit Python ausgeführt werden (Bibliothek [`requests`](https://pypi.org/project/requests/)) und die Antwort von der API mit einem JSON-Parser (`json`) gelesen werden. Eine andere Möglichkeit ist, den Solr-Client [`pysolr`](https://pypi.org/project/pysolr/) zu benutzen. Dieser muss zunächst in der Umgebung mit `pip install -q pysolr` (oder ggf. mit Conda: `conda install conda-forge::pysolr`) installiert werden.

In [1]:
# Python-Bibliothek pysolr installieren
%pip install -q pysolr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


### KI-generierter Programmcode

Die Erstellung des Python-Programmcodes ist KI-gestützt möglich. Die folgenden KI-Prompts sind mit ChatGPT (GPT-4o) erfolgreich getestet und liefern (oft 😉) das gewünschte Ergebnis.

<div class="alert alert-block alert-info">
<b>Prompt:</b> Ich möchte in Python mit dem Solr-Client <code>pysolr</code> auf den Endpunkt <code>https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper</code> zugreifen. Kannst Du mir einen Python-Code erstellen, der im Feld <code>location</code> nach „Buenos Aires“ sucht und auch <code>hasLoadedIssues</code> auf „wahr“ setzt. Gibt bitte <code>id</code>, <code>title</code>, <code>location</code>, <code>frequency</code> und <code>progress</code> für jeden Suchtreffer aus.
</div>

In [3]:
import pysolr

# Solr-Endpunkt-URL
solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper'

# Solr-Client initialisieren
solr = pysolr.Solr(solr_url, timeout=10)

# Suchparameter
q = {
    'q': 'location:"Buenos Aires" AND hasLoadedIssues:true',
    'fl': 'id,title,location,frequency,progress',
    'rows': 100  # Anzahl der zurückzugebenden Ergebnisse (hier auf 100 gesetzt, anpassbar)
}

# Suche ausführen
results = solr.search(**q)

# Ergebnisse ausgeben
if results.hits > 0:
    print(f"Anzahl der Suchtreffer: {results.hits}\n")
    for result in results:
        print(f"ID: {result.get('id', 'N/A')}")
        print(f"Title: {result.get('title', 'N/A')}")
        print(f"Location: {result.get('location', 'N/A')}")
        print(f"Frequency: {result.get('frequency', 'N/A')}")
        print(f"Progress: {result.get('progress', 'N/A')}")
        print("---" * 10)
else:
    print("Keine Ergebnisse für die angegebene Suchanfrage gefunden.")

Anzahl der Suchtreffer: 2

ID: 2232340-5
Title: ['Das andere Deutschland']
Location: ['Buenos Aires']
Frequency: ['http://id.loc.gov/vocabulary/frequencies/mon']
Progress: ['[1.]1937/38,Juli-Dez.=Nr. 3-8; 2.1939,Jan. - 5.1941,Dez. = Nr. 9-45; mehr nicht digitalisiert']
------------------------------
ID: 2149754-0
Title: ['La otra Alemania']
Location: ['Montevideo', 'Buenos Aires']
Frequency: ['http://id.loc.gov/vocabulary/frequencies/irr']
Progress: ['5.1942,Jan. - 6.1943,15.Dez. = Nr. 46-79; [N.F.] 1.1944,1/2(Febr.)-3/4(März); 7.1944/45,Apr.-15.Dez.=Nr. 80/81-108; 8.1946,1.Jan.-15.Aug.=Nr. 109-124; 1946,1.Sept. - 1949,Jan. = Nr. 125-175; damit Ersch. eingest.']
------------------------------


In [ ]:
import pysolr

# Solr-Endpunkt-URL
solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper'

# Solr-Client initialisieren
solr = pysolr.Solr(solr_url, timeout=10)

# Suchparameter
q = {
    'q': 'title:"Rastatter Tageblatt" AND hasLoadedIssues:true', #Zeitungstitel im Zeitungsportal: https://www.deutsche-digitale-bibliothek.de/newspaper/select/title
    'fl': 'id,title,location,frequency,progress',
    'rows': 10  # Anzahl der zurückzugebenden Ergebnisse
}

# Suche ausführen
results = solr.search(**q)

# Ergebnisse ausgeben
for result in results:
    print(f"ID: {result.get('id', 'N/A')}")
    print(f"Title: {result.get('title', 'N/A')}")
    print(f"Location: {result.get('location', 'N/A')}")
    print(f"Frequency: {result.get('frequency', 'N/A')}")
    print(f"Progress: {result.get('progress', 'N/A')}")
    print("-" * 40)

ID: 3091248-9
Title: ['Rastatter Tageblatt']
Location: ['Rastatt']
Frequency: ['http://id.loc.gov/vocabulary/frequencies/dyl']
Progress: ['139. Jahrgang, Nummer 205 (2. September 1942) [?]-140. Jahrgang, Nummer 100 (30. April 1943) ; damit  Erscheinen eingestellt']
----------------------------------------


## Suchindex `newspaper-issues`

Der Suchindex `newspaper-issues` ist ein weiterer Suchindex des Zeitungsportals. Dieser enthält alle Ausgaben (`type:issue`) einer Zeitung und alle Seiten (`type:page`). Wenn man nur in einem Zeitungstitel suchen möchte, so kann dies über `zdb_id:{ID der Zeitschriftendatenbank}` (für die „La otra Alemania“ ist es `zdb_id:2149754-0`) eingegrenzt werden.

### KI-generierter Programmcode

<div class="alert alert-block alert-info">
<b>Prompt:</b> Schreibe ein Python-Code, der mithilfe der <code>pysolr</code>-Bibliothek eine Suche in einem Solr-Index durchführt und die Ergebnisse in ein Pandas DataFrame überführt. Der Solr-Index ist über die URL <code>https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues</code> erreichbar. Die Suchabfrage soll nach Dokumenten mit der <code>zdb_id</code> „2149754-0“ und dem <code>type</code> „issue“ suchen und bis zu 1000 Ergebnisse zurückgeben. Anschließend sollen die Ergebnisse in ein Pandas DataFrame überführt und angezeigt werden.
</div>

In [4]:
import pysolr
import pandas as pd

# Solr-Endpunkt-URL
solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues'

# Solr-Client initialisieren
solr = pysolr.Solr(solr_url, timeout=10)

# Suchparameter
q = {
    'q': 'zdb_id:2149754-0 AND type:issue',
    'rows': 1000  # Anzahl der zurückzugebenden Ergebnisse (hier auf 1000 gesetzt)
}

# Suche ausführen
results = solr.search(**q)

# Überführen der Ergebnisse in ein Pandas DataFrame
df = pd.DataFrame(results.docs)

# DataFrame anzeigen
print(df)

                                   id  \
0    AK26JXHRHC2T72IMFGUAJ7EIFJQLY4CL   
1    XFL3GU2ZVC4YMZB2LUIPGODKASCB7X3K   
2    GCADKIMUKSSL23LL6BXHWRNP66KQR37R   
3    XH5D7MEXRH5MFIYJBCNTTE4SUAWPKYQ3   
4    TJ6BV7NK6JA62D6NDUXQHCYJCN3YNHLO   
..                                ...   
124  MMJX3P3AFQO2ZJT3DPNTKI2E7TNTM67B   
125  MJLH3WN4QS47AML3NKLTLG6WJVVPQRAF   
126  PBIKLXIOC4YIVKWLH5YGQ4V5LVX7TCV5   
127  6JQ7GVLHN7FMK4THCWQN3ZDAVT2CLK56   
128  LVQZNU2GQY4R3QSFPFEHBHEINPS2SX3P   

                                           paper_title  \
0    La otra Alemania : órgano de los alemanes demo...   
1    La otra Alemania : órgano de los alemanes demo...   
2    La otra Alemania : órgano de los alemanes demo...   
3    La otra Alemania : órgano de los alemanes demo...   
4    La otra Alemania : órgano de los alemanes demo...   
..                                                 ...   
124  La otra Alemania : órgano de los alemanes demo...   
125  La otra Alemania : órgano de los alema

In [5]:
import pandas as pd

solr_url = 'https://api.deutsche-digitale-bibliothek.de/2/search/index/newspaper-issues'
solr = pysolr.Solr(solr_url, always_commit=True, timeout=100)

q = {
    'q': 'zdb_id:3091248-9 AND type:page AND publication_date:[1942-01-01T00:00:00Z TO 1943-12-31T23:59:59Z] AND plainpagefulltext: erdbeben*',
    'rows': 1000
}

response = solr.search(**q)

# Überführen der Ergebnisse in ein Pandas DataFrame
df = pd.DataFrame(response.docs)

# DataFrame anzeigen
print(df)

                                                   id  pagenumber  \
0   ZS7H6FDKXDNVPBOX4MVNAT66UNIULPIS-ALTO6951586_D...           3   
1   3I6RUQKRENRKTVKKB3GCNKCUQGZGTOVC-ALTO6951637_D...           6   
2   U7AUPHJQK3FN7MDFTT27BJAWX564G34L-ALTO6951495_D...           4   
3   L7KAFMAV275IF6RL5VEFRGRVHIBUOMLO-ALTO6951621_D...           2   
4   L7KAFMAV275IF6RL5VEFRGRVHIBUOMLO-ALTO6951623_D...           4   
5   JTHS6JJDMWCF5A6GG53R25CWXKZHGTWQ-ALTO6951617_D...           2   
6   XDA64WA6F74B57Z3IJBGPTSCLVQERTNO-ALTO6951395_D...           4   
7   3L2E3PBA2FEETMWLULTG3GJL2ER7CPVU-ALTO6951367_D...           4   
8   AJRCPEBZAJX74O3LWVRIRZWAIW44CYVR-ALTO6951601_D...           2   
9   3PPZONPWNWONRHPZWSO7M6VF6J3PZ2DO-ALTO6951597_D...           2   
10  B4BQV7IOGYEBAV2DGGSVVWCBTQH3XKFY-ALTO6951399_D...           4   
11  RXA3UVXJYBLRMZ7M2QKUZJDN6AXI4QQN-ALTO6951423_D...           4   
12  AUB6FPO62MVCCD7PNCJZ3ZFTRULVA3NF-ALTO6951449_D...           2   
13  HBIHHEWR3FDI2WT2JEXAZD6YOUUVHU

In [6]:
df.to_excel('results.xlsx')

### Erste Datenanalyse

In dem Dataframe können nun Datenanalysen vorgenommen werden. Wir wollen den Publikationszeitraum von der Zeitung „La otra Alemania“ ermitteln.

In [7]:
# Sicherstellen, dass publication_date als Datumswerte formatiert sind
df['publication_date'] = pd.to_datetime(df['publication_date'])

# Frühestes und spätestes Datum ermitteln
earliest_date = df['publication_date'].min()
latest_date = df['publication_date'].max()

# Ergebnisse anzeigen
print(f"Frühestes Veröffentlichungsdatum: {earliest_date}")
print(f"Spätestes Veröffentlichungsdatum: {latest_date}")

Frühestes Veröffentlichungsdatum: 1942-11-30 12:00:00+00:00
Spätestes Veröffentlichungsdatum: 1943-04-30 12:00:00+00:00
